# Load Libraries

In [1]:
# pip3 install s3fs
from sagemaker import Session
import sagemaker
import boto3
import re
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import os

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
role = get_execution_role()

bucket_name = 'eliezerraj-908671954593-dataset'
prefix_name = 'customer/notebook/output'
output_path = f's3://{bucket_name}/{prefix_name}'
data_location = f"s3://{bucket_name}/{prefix_name}/train"

train_data_file = f's3://{bucket_name}/{prefix_name}/train/train_data.csv'

print("---------------------------------")
print(f"bucket_name '{bucket_name}':")
print(f"prefix_name '{prefix_name}':")
print(f"data_location '{data_location}':")
print(f"output_path '{output_path}':")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
---------------------------------
bucket_name 'eliezerraj-908671954593-dataset':
prefix_name 'customer/notebook/output':
data_location 's3://eliezerraj-908671954593-dataset/customer/notebook/output/train':
output_path 's3://eliezerraj-908671954593-dataset/customer/notebook/output':


In [3]:
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()

#df_data = pd.read_csv(train_data_file, skiprows = 1, header=None)
#df_data_scaled = scaler.fit_transform(df_data).astype('float32')
#df_data_scaled

# Id the dataset is already scaled
df_data = pd.read_csv(train_data_file, header=None)
df_data

,0,1,2,3
0,45.0,3.0,1.0,3.0
1,49.0,5.0,3.0,1.0
2,51.0,3.0,3.0,4.0
3,40.0,4.0,1.0,1.0
4,40.0,3.0,0.0,3.0
...,...,...,...,...
10122,50.0,2.0,3.0,2.0
10123,41.0,2.0,0.0,2.0
10124,44.0,1.0,1.0,1.0
10125,30.0,2.0,3.0,2.0


In [4]:
df_data_k = df_data.astype('float32').to_numpy()
df_data_k

array([[45.,  3.,  1.,  3.],
       [49.,  5.,  3.,  1.],
       [51.,  3.,  3.,  4.],
       ...,
       [44.,  1.,  1.,  1.],
       [30.,  2.,  3.,  2.],
       [43.,  2.,  3.,  1.]], dtype=float32)

# Build Model

In [5]:
from sagemaker import KMeans

kmeans = KMeans(role=role,
                instance_count=1,
                instance_type='ml.m4.xlarge',
                data_location=data_location,
                output_path=output_path,
                init_method='kmeans++',
                k=5,
                epochs=5)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [6]:
kmeans.fit(kmeans.record_set(df_data_k))

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: kmeans-2024-04-29-00-04-39-283


2024-04-29 00:04:39 Starting - Starting the training job...
2024-04-29 00:04:53 Starting - Preparing the instances for training...
2024-04-29 00:05:27 Downloading - Downloading input data...
2024-04-29 00:05:57 Downloading - Downloading the training image.........
2024-04-29 00:07:37 Training - Training image download completed. Training in progress.
2024-04-29 00:07:37 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
Running default environment configuration script
[04/29/2024 00:07:32 INFO 140116863772480] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'init_method': 'random', 'mini_batch_size': '5000', 'epochs': '1', 'extra_center_factor': 'auto', 'local_lloyd_max_iter': '300', 'local_lloyd_tol': '0.0001', 'local_lloyd_init_method': 'kmeans++', 'local_lloyd_num_trials': 'auto', 'half_life_time_size': '0', 'eval_metrics': '["msd"]', 'force_dense': 'true', '_disable_wait_

# NOT WORK WITH INFERENCE - Deploy model - create endpoint

In [35]:
# Get folder model.tar.z

job_name = kmeans.latest_training_job.name
model_output_path = f's3://{bucket_name}/{prefix_name}/{job_name}/output/model.tar.gz'

print("model_output_path : ", model_output_path)

model_output_path :  s3://eliezerraj-908671954593-dataset/customer/notebook/output/kmeans-2024-04-28-21-42-39-435/output/model.tar.gz


In [36]:
# Set the model name
from time import gmtime, strftime

client = boto3.client(service_name="sagemaker")
region = boto3.Session().region_name

model_name = "kmeans-serverless-customer-v2-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Model name: " + model_name)

Model name: kmeans-serverless-customer-v2-2024-04-28-21-51-20


In [37]:
# Get the container image
from sagemaker import image_uris
container = image_uris.retrieve("kmeans", region, "latest")

print("containere: " + container)

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


containere: 404615174143.dkr.ecr.us-east-2.amazonaws.com/kmeans:1


In [38]:
# Set environment variables
byo_container_env_vars = {"SAGEMAKER_CONTAINER_LOG_LEVEL": "20"}

create_model_response = client.create_model(
    ModelName=model_name,
    Containers=[
        {
            "Image": container,
            "Mode": "SingleModel",
            "ModelDataUrl": model_output_path,
            "Environment": byo_container_env_vars,
        }
    ],
    ExecutionRoleArn=role,
)

print("Model Arn: " + create_model_response["ModelArn"])

Model Arn: arn:aws:sagemaker:us-east-2:908671954593:model/kmeans-serverless-customer-v2-2024-04-28-21-51-20


# NOT WORK WITH INFERENCE Serverless endpoint_config

In [39]:
# Serveless endpoint config
model_epc_name = "mlops-serverless-epc-customer-v2-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=model_epc_name,
    ProductionVariants=[
        {
            "VariantName": "byoVariant",
            "ModelName": model_name,
            "ServerlessConfig": {
                "MemorySizeInMB": 4096,
                "MaxConcurrency": 1,
            },
        },
    ],
)

print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])

Endpoint Configuration Arn: arn:aws:sagemaker:us-east-2:908671954593:endpoint-config/mlops-serverless-epc-customer-v2-2024-04-28-21-51-59


In [40]:
# Serveless endpoint
endpoint_name = "kmeans-serverless-ep-customer-v2-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=model_epc_name,
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

Endpoint Arn: arn:aws:sagemaker:us-east-2:908671954593:endpoint/kmeans-serverless-ep-customer-v2-2024-04-28-21-52-17


# Deploy INFERENCE EC2 Model, Endpoint Config and Endpoint

In [7]:
# EC2 endpoint
kmeans_predictor = kmeans.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating model with name: kmeans-2024-04-29-00-08-37-923
INFO:sagemaker:Creating endpoint-config with name kmeans-2024-04-29-00-08-37-923
INFO:sagemaker:Creating endpoint with name kmeans-2024-04-29-00-08-37-923


------!

NameError: name 'endpoint_config_response' is not defined

In [9]:
# Install model
kmeans_predictor.serializer = sagemaker.serializers.CSVSerializer()

# Inference Validation

In [53]:
print("df_data : ", df_data)

result = kmeans_predictor.predict(df_data)
clusters = [r.label["closest_cluster"].float32_tensor.values[0] for r in result]

print("===> (CLUSTER )result for 4 posicion : ", clusters[0:4])

df_data :            0    1    2    3
0      45.0  3.0  1.0  3.0
1      49.0  5.0  3.0  1.0
2      51.0  3.0  3.0  4.0
3      40.0  4.0  1.0  1.0
4      40.0  3.0  0.0  3.0
...     ...  ...  ...  ...
10122  50.0  2.0  3.0  2.0
10123  41.0  2.0  0.0  2.0
10124  44.0  1.0  1.0  1.0
10125  30.0  2.0  3.0  2.0
10126  43.0  2.0  3.0  1.0

[10127 rows x 4 columns]
===> (CLUSTER )result for 4 posicion :  [2.0, 1.0, 1.0, 1.0]


In [54]:
df_result_final = pd.read_csv(train_data_file, names=['Customer_Age', 
                                                      'Dependent_count', 
                                                      'Education_Level_Quality',
                                                      'Income_Category_Quality'])
df_result_final["CLUSTER"] = clusters
df_result_final.head(100)

,Customer_Age,Dependent_count,Education_Level_Quality,Income_Category_Quality,CLUSTER
0,45.0,3.0,1.0,3.0,2.0
1,49.0,5.0,3.0,1.0,1.0
2,51.0,3.0,3.0,4.0,1.0
3,40.0,4.0,1.0,1.0,1.0
4,40.0,3.0,0.0,3.0,1.0
...,...,...,...,...,...
95,64.0,1.0,3.0,1.0,1.0
96,45.0,3.0,3.0,2.0,2.0
97,53.0,3.0,5.0,2.0,1.0
98,56.0,3.0,2.0,5.0,1.0


# Add a new data

In [81]:
# new data customes

df_new_customer_data = [{'Customer_Age': 40, 
                         'Dependent_count': 3, 
                         'Education_Level_Quality': 0, 
                         'Income_Category_Quality': 3 }]

df_new_customer_data = pd.DataFrame(df_new_customer_data)
df_new_customer_data

,Customer_Age,Dependent_count,Education_Level_Quality,Income_Category_Quality
0,40,3,0,3


In [82]:
df_new_customer_data.columns=df_data.columns
df_customer_test = pd.concat([df_new_customer_data, df_data], axis=0)
df_customer_test.head(3)

,0,1,2,3
0,40.0,3.0,0.0,3.0
0,45.0,3.0,1.0,3.0
1,49.0,5.0,3.0,1.0


In [83]:
new_customer_result_02 = kmeans_predictor.predict(df_customer_test)
new_customer_clusters_02 = [r.label["closest_cluster"].float32_tensor.values[0] for r in new_customer_result_02]

In [84]:
df_customer_test["CLUSTER"] = new_customer_clusters_02
df_customer_test.head(11)

,0,1,2,3,CLUSTER
0,40.0,3.0,0.0,3.0,1.0
0,45.0,3.0,1.0,3.0,2.0
1,49.0,5.0,3.0,1.0,1.0
2,51.0,3.0,3.0,4.0,1.0
3,40.0,4.0,1.0,1.0,1.0
4,40.0,3.0,0.0,3.0,1.0
5,44.0,2.0,3.0,2.0,1.0
6,51.0,4.0,0.0,5.0,1.0
7,32.0,0.0,1.0,3.0,1.0
8,37.0,3.0,0.0,3.0,1.0


In [86]:
print(kmeans_predictor.endpoint)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


kmeans-2024-04-29-00-08-37-923


In [87]:
kmeans_predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: kmeans-2024-04-29-00-08-37-923
INFO:sagemaker:Deleting endpoint with name: kmeans-2024-04-29-00-08-37-923
